In [ ]:
import tensorflow as tf
import numpy as np
import pathlib

In [ ]:
data_dir = pathlib.Path("data")

count = len(list(data_dir.glob("*/*.jpg")))
print(f"Found {count} images")

In [ ]:
batch_size = 32
img_height = 220
img_width = 220

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=1,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

In [ ]:
def build_cnn_model():
    return tf.keras.Sequential(
        [
            tf.keras.layers.Rescaling(1.0 / 255),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(2, activation="softmax"),
        ]
    )


model = build_cnn_model()

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
BATCH_SIZE = 12
EPOCHS = 12

model.fit(train_ds, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=val_ds)

In [ ]:
_, acc = model.evaluate(val_ds)
print(acc)

In [ ]:
import gradio as gr


def classify(image):
    image = image.reshape((-1, 220, 220, 3))
    prediction = model.predict(image).flatten()
    return {"Cat": float(prediction[0]), "Dog": float(prediction[1])}


gr.Interface(
    fn=classify, inputs=gr.Image(shape=(220, 220)), outputs=gr.Label(num_top_classes=2)
).launch()